In [1]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = 'gpt-4o-mini')

In [14]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

class EmailParser(BaseModel):
    sender_name: str = Field(description='이메일 송신자') # 보낸사람
    title: str = Field(description='이메일 제목') # 제목
    content: str = Field(description='이메일 제목') # 내용
    
parser = JsonOutputParser(pydantic_object=EmailParser)
parser

JsonOutputParser(pydantic_object=<class '__main__.EmailParser'>)

In [16]:
prompt_temp = PromptTemplate(
    template='이메일을 읽고 정보를 추출하세요,\n{format_instructions}\n{email}',
    input_variables=['email'],
    partial_variables={'format_instructions':parser.get_format_instructions()}
)

prompt_temp

PromptTemplate(input_variables=['email'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"sender_name": {"description": "이메일 송신자", "title": "Sender Name", "type": "string"}, "title": {"description": "이메일 제목", "title": "Title", "type": "string"}, "content": {"description": "이메일 제목", "title": "Content", "type": "string"}}, "required": ["sender_name", "title", "content"]}\n```'}, template='이메일을 읽고 정보를 추출하세요,\n{format_instructions}\n{email}')

In [10]:
prompt = prompt_temp.format(email = '보내는 사람: 김철수\n제목: 금일자회의록(9/8)\n회의내용: 오늘 00팀과 진행힌 회의록 입니다')
prompt

'이메일을 읽고 정보를 추출하세요.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"sender": {"description": "이메일 보낸 사람", "title": "Sender", "type": "string"}, "subject": {"description": "이메일 제목", "title": "Subject", "type": "string"}}, "required": ["sender", "subject"]}\n```\n\n보내는 사람: 김철수\n제목: 금일자회의록(9/8)\n회의내용: 오늘 00팀과 진행힌 회의록 입니다'

In [11]:
response = llm.invoke(prompt)
response

AIMessage(content='```json\n{"sender":"김철수","subject":"금일자회의록(9/8)"}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 234, 'total_tokens': 258, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CDMjM7SEjMTKKff59LXWuBTLbY7dC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--74ade4d4-af20-424e-9771-5d195888ca07-0', usage_metadata={'input_tokens': 234, 'output_tokens': 24, 'total_tokens': 258, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})